In [1]:


import os
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# GPU CONFIGURATION - MAXIMUM P100 UTILIZATION

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    # Maximum P100 optimization
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.enabled = True
    
    # Enable TF32 on Ampere GPUs (no effect on P100 but future-proof)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    
    # Set max split size to avoid fragmentation
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
    
    print("P100 GPU fully optimized for maximum throughput")

# MODEL ARCHITECTURES

class MicroArtifactCNN(nn.Module):
    """Specialized for high-frequency synthesis artifacts"""
    
    def __init__(self, dropout=0.6):
        super().__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(dropout * 0.3)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(dropout * 0.4)
        )

        
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(dropout * 0.5)
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(dropout)
        )
        
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 1)
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.classifier(x)
        return x


class XceptionDeepfake(nn.Module):
    """EfficientNet-based depthwise separable architecture"""
    
    def __init__(self, dropout=0.6):
        super().__init__()
        
        self.entry = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.sep_block1 = self._make_sep_block(64, 128, dropout * 0.4)
        self.sep_block2 = self._make_sep_block(128, 256, dropout * 0.5)
        self.sep_block3 = self._make_sep_block(256, 512, dropout * 0.6)
        
        self.exit = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 1)
        )
        
    def _make_sep_block(self, in_ch, out_ch, dropout):
        return nn.Sequential(
            nn.Conv2d(in_ch, in_ch, 3, padding=1, groups=in_ch),
            nn.BatchNorm2d(in_ch),
            nn.ReLU(),
            nn.Conv2d(in_ch, out_ch, 1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(dropout)
        )
    
    def forward(self, x):
        x = self.entry(x)
        x = self.sep_block1(x)
        x = self.sep_block2(x)
        x = self.sep_block3(x)
        x = self.exit(x)
        return x


class ResNet50Deepfake(nn.Module):
    """ResNet architecture tuned for artifact detection"""
    
    def __init__(self, dropout=0.6):
        super().__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2, padding=1)
        )
        
        self.layer1 = self._make_layer(64, 64, 3, dropout * 0.3)
        self.layer2 = self._make_layer(64, 128, 4, dropout * 0.4)
        self.layer3 = self._make_layer(128, 256, 6, dropout * 0.5)
        self.layer4 = self._make_layer(256, 512, 3, dropout * 0.6)
        
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 1)
        )
        
    def _make_layer(self, in_ch, out_ch, blocks, dropout):
        layers = []
        for i in range(blocks):
            layers.append(nn.Conv2d(in_ch if i == 0 else out_ch, out_ch, 3, padding=1))
            layers.append(nn.BatchNorm2d(out_ch))
            layers.append(nn.ReLU())
            if i == blocks - 1:
                layers.append(nn.MaxPool2d(2))
                layers.append(nn.Dropout2d(dropout))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.classifier(x)
        return x


class EfficientNetB0Deepfake(nn.Module):
    """EfficientNet-B0 for deepfake detection"""
    
    def __init__(self, dropout=0.6):
        super().__init__()
        
        # Stem
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU()
        )
        
        # MBConv blocks
        self.blocks = nn.Sequential(
            self._make_mbconv(32, 16, 1, 1, dropout * 0.2),
            self._make_mbconv(16, 24, 2, 6, dropout * 0.3),
            self._make_mbconv(24, 40, 2, 6, dropout * 0.4),
            self._make_mbconv(40, 80, 3, 6, dropout * 0.5),
            self._make_mbconv(80, 112, 3, 6, dropout * 0.5),
            self._make_mbconv(112, 192, 4, 6, dropout * 0.6),
            self._make_mbconv(192, 320, 1, 6, dropout * 0.6)
        )
        
        # Head
        self.head = nn.Sequential(
            nn.Conv2d(320, 1280, 1),
            nn.BatchNorm2d(1280),
            nn.SiLU(),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Dropout(dropout),
            nn.Linear(1280, 1)
        )
        
    def _make_mbconv(self, in_ch, out_ch, stride, expand, dropout):
        mid_ch = in_ch * expand
        return nn.Sequential(
            nn.Conv2d(in_ch, mid_ch, 1),
            nn.BatchNorm2d(mid_ch),
            nn.SiLU(),
            nn.Conv2d(mid_ch, mid_ch, 3, stride=stride, padding=1, groups=mid_ch),
            nn.BatchNorm2d(mid_ch),
            nn.SiLU(),
            nn.Conv2d(mid_ch, out_ch, 1),
            nn.BatchNorm2d(out_ch),
            nn.Dropout2d(dropout)
        )
    
    def forward(self, x):
        x = self.stem(x)
        x = self.blocks(x)
        x = self.head(x)
        return x


# FACE DETECTION & EXTRACTION WITH SURROUNDING CONTEXT

def extract_face_with_context(frame, detector, expand_ratio=1.5):
    """
    Extract face with surrounding region (jawline, neck, hairline)
    expand_ratio: 1.5 means 50% expansion around detected face
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(80, 80))
    
    if len(faces) == 0:
        return None
    
    # Get largest face
    x, y, w, h = max(faces, key=lambda f: f[2] * f[3])
    
    # Expand to include surrounding context
    center_x, center_y = x + w // 2, y + h // 2
    new_w = int(w * expand_ratio)
    new_h = int(h * expand_ratio)
    
    x1 = max(0, center_x - new_w // 2)
    y1 = max(0, center_y - new_h // 2)
    x2 = min(frame.shape[1], center_x + new_w // 2)
    y2 = min(frame.shape[0], center_y + new_h // 2)
    
    face_region = frame[y1:y2, x1:x2]
    
    return face_region if face_region.size > 0 else None

# VIDEO TO FRAMES EXTRACTION

def extract_frames_from_video(video_path, detector, max_frames=30, fps_sample=1, face_expansion_ratio=1.5):
    """
    Extract frames with faces from video
    max_frames: maximum frames to extract per video
    fps_sample: sample every N frames to maintain temporal order
    face_expansion_ratio: expansion ratio for face context
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    frames = []
    frame_idx = 0
    extracted = 0
    
    # Sample uniformly across video duration
    sample_interval = max(1, total_frames // max_frames)
    
    while cap.isOpened() and extracted < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_idx % (sample_interval * fps_sample) == 0:
            face = extract_face_with_context(frame, detector, expand_ratio=face_expansion_ratio)
            if face is not None:
                frames.append(face)
                extracted += 1
        
        frame_idx += 1
    
    cap.release()
    return frames

# DATASET CLASS

class DeepfakeDataset(Dataset):
    def __init__(self, frames, labels, transform=None):
        self.frames = frames
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.frames)
    
    def __getitem__(self, idx):
        frame = self.frames[idx]
        label = self.labels[idx]
        
        if self.transform:
            augmented = self.transform(image=frame)
            frame = augmented['image']
        
        return frame, torch.tensor(label, dtype=torch.float32)

# DATA AUGMENTATION - PRESERVING ARTIFACTS

def get_transforms(img_size=224):
    """
    Augmentations that preserve manipulation artifacts
    """
    train_transform = A.Compose([
        A.Resize(img_size, img_size),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=5, p=0.3),
        A.OneOf([
            A.GaussNoise(var_limit=(5.0, 15.0), p=1.0),
            A.ISONoise(p=1.0),
        ], p=0.2),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    
    val_transform = A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    
    return train_transform, val_transform

# TRAINING FUNCTION
def train_model(model, train_loader, val_loader, epochs=25, lr=1e-4, model_name="model"):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)
    
    scaler = torch.cuda.amp.GradScaler()
    
    best_val_acc = 0.0
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        for frames, labels in pbar:
            frames, labels = frames.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            
            optimizer.zero_grad(set_to_none=True)
            
            with torch.cuda.amp.autocast():
                outputs = model(frames).squeeze()
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            with torch.no_grad():
                preds = (torch.sigmoid(outputs) > 0.5).float()
                train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)
            
            pbar.set_postfix({'loss': f'{loss.item():.4f}', 'acc': f'{train_correct/train_total:.4f}'})
        
        avg_train_loss = train_loss / len(train_loader)
        train_acc = train_correct / train_total
        train_losses.append(avg_train_loss)
        train_accs.append(train_acc)
        
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for frames, labels in val_loader:
                frames, labels = frames.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                
                with torch.cuda.amp.autocast():
                    outputs = model(frames).squeeze()
                    loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                preds = (torch.sigmoid(outputs) > 0.5).float()
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
        
        avg_val_loss = val_loss / len(val_loader)
        val_acc = val_correct / val_total
        val_losses.append(avg_val_loss)
        val_accs.append(val_acc)
        
        print(f"\nEpoch {epoch+1}: Train Loss={avg_train_loss:.4f}, Train Acc={train_acc:.4f} | "
              f"Val Loss={avg_val_loss:.4f}, Val Acc={val_acc:.4f}")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
            }, f'{model_name}_best.pth')
            print(f"Saved best model: {model_name}_best.pth (Val Acc: {val_acc:.4f})")
        
        scheduler.step()
        
        if (epoch + 1) % 5 == 0:
            torch.cuda.empty_cache()
    
    return train_losses, val_losses, train_accs, val_accs, best_val_acc
# MAIN EXECUTION

def main():
    print("Searching for dataset directories...")
    
    possible_roots = [
        "/kaggle/input/deep-fake-detection-dfd-entire-original-dataset",
        "/kaggle/input/*"
    ]
    
    REAL_DIR = None
    FAKE_DIR = None
    
    for root in possible_roots:
        real_patterns = [
            f"{root}/DFD_original sequences",
            f"{root}/DFD_original_sequences",
            f"{root}/original_sequences",
            f"{root}/Real video"
        ]
        
        for pattern in real_patterns:
            matches = glob.glob(pattern)
            if matches:
                REAL_DIR = matches[0]
                print(f"Found REAL directory: {REAL_DIR}")
                break
        
        fake_patterns = [
            f"{root}/DFD_manipulated sequences/DFD_manipulated sequences",
            f"{root}/DFD_manipulated_sequences/DFD_manipulated_sequences",
            f"{root}/DFD_manipulated sequences",
            f"{root}/Deepfake video"
        ]
        
        for pattern in fake_patterns:
            matches = glob.glob(pattern)
            if matches:
                FAKE_DIR = matches[0]
                print(f"Found FAKE directory: {FAKE_DIR}")
                break
        
        if REAL_DIR and FAKE_DIR:
            break
    
    if not REAL_DIR or not FAKE_DIR:
        print("\n  Could not auto-detect dataset directories!")
        print("\n Available directories in /kaggle/input:")
        for item in glob.glob("/kaggle/input/*"):
            print(f"   {item}")
            if os.path.isdir(item):
                for subitem in glob.glob(f"{item}/*")[:10]:
                    print(f"      └─ {os.path.basename(subitem)}")
        
        raise FileNotFoundError(
            "Dataset directories not found. Please check the directory structure above "
            "and update REAL_DIR and FAKE_DIR manually in the code."
        )
    
    # =================== CUSTOM SETTINGS ===================
    FPS_SAMPLE = 3
    MAX_FRAMES_PER_VIDEO = 200
    FACE_EXPANSION_RATIO = 1.7
    TARGET_SIZE = (224, 224)
    IMG_SIZE = 224
    BATCH_SIZE = 222
    EPOCHS = 50
    # ========================================================
    
    print(" Initializing Haar Cascade face detector...")
    detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    print("Haar Cascade detector initialized")
    
    # Training on persons 01 through 10
    TRAIN_ON_SPECIFIC_PERSONS = True
    PERSON_IDS = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]
    
    all_real_videos = [f for f in os.listdir(REAL_DIR) if f.endswith('.mp4')]
    all_fake_videos = [f for f in os.listdir(FAKE_DIR) if f.endswith('.mp4')]
    
    if TRAIN_ON_SPECIFIC_PERSONS:
        print(f"  Training on Person IDs: {', '.join(PERSON_IDS)}")
        real_videos = [os.path.join(REAL_DIR, f) for f in all_real_videos 
                       if any(f.startswith(f'{pid}_') for pid in PERSON_IDS)]
        fake_videos = [os.path.join(FAKE_DIR, f) for f in all_fake_videos 
                       if any(f.startswith(f'{pid}_') for pid in PERSON_IDS)]
    else:
        print(f"  Training on ALL person IDs")
        real_videos = [os.path.join(REAL_DIR, f) for f in all_real_videos]
        fake_videos = [os.path.join(FAKE_DIR, f) for f in all_fake_videos]
    
    print(f"\nDataset Statistics:")
    print(f"   Total videos in REAL dir: {len(all_real_videos)}")
    print(f"   Total videos in FAKE dir: {len(all_fake_videos)}")
    if TRAIN_ON_SPECIFIC_PERSONS:
        print(f"  Filtered REAL videos (Persons {', '.join(PERSON_IDS)}): {len(real_videos)}")
        print(f"  Filtered FAKE videos (Persons {', '.join(PERSON_IDS)}): {len(fake_videos)}")
    else:
        print(f"  Using ALL REAL videos: {len(real_videos)}")
        print(f"  Using ALL FAKE videos: {len(fake_videos)}")
    
    assert len(real_videos) > 0, " No real videos found! Check video naming convention."
    assert len(fake_videos) > 0, " No fake videos found! Check video naming convention."
    
    print("\n Extracting frames from videos using Haar Cascade...")
    all_frames = []
    all_labels = []
    
    # Real videos
    for video_path in tqdm(real_videos, desc="Processing REAL videos"):
        frames = extract_frames_from_video(video_path, detector, MAX_FRAMES_PER_VIDEO, FPS_SAMPLE, FACE_EXPANSION_RATIO)
        frames = [cv2.resize(f, TARGET_SIZE) for f in frames if f is not None]
        all_frames.extend(frames)
        all_labels.extend([0] * len(frames))
    
    # Fake videos
    for video_path in tqdm(fake_videos, desc="Processing FAKE videos"):
        frames = extract_frames_from_video(video_path, detector, MAX_FRAMES_PER_VIDEO, FPS_SAMPLE, FACE_EXPANSION_RATIO)
        frames = [cv2.resize(f, TARGET_SIZE) for f in frames if f is not None]
        all_frames.extend(frames)
        all_labels.extend([1] * len(frames++++++++++++++++++++++++++++++++++++++++++d
    
    # =================== DATA BALANCING ===================
    real_count = sum(1 for l in all_labels if l == 0)
    fake_count = sum(1 for l in all_labels if l == 1)
    min_count = min(real_count, fake_count)
    
    print(f"\n DATA BALANCING ACTIVE")
    print(f"   Before: Real={real_count}, Fake={fake_count} (Imbalance ratio: {max(real_count, fake_count)/min_count:.2f}:1)")
    print(f"   Balancing to: {min_count} samples per class...")
    
    real_indices = [i for i, l in enumerate(all_labels) if l == 0]
    fake_indices = [i for i, l in enumerate(all_labels) if l == 1]
    
    np.random.seed(42)
    np.random.shuffle(real_indices)
    np.random.shuffle(fake_indices)
    
    real_indices = real_indices[:min_count]
    fake_indices = fake_indices[:min_count]
    balanced_indices = real_indices + fake_indices
    
    all_frames = [all_frames[i] for i in balanced_indices]
    all_labels = [all_labels[i] for i in balanced_indices]
    
    final_real = sum(1 for l in all_labels if l == 0)
    final_fake = sum(1 for l in all_labels if l == 1)
    print(f"   After: Real={final_real}, Fake={final_fake} (Perfect 1:1 balance)")
    
    train_frames, val_frames, train_labels, val_labels = train_test_split(
        all_frames, all_labels, test_size=0.2, stratify=all_labels, random_state=42
    )
    
    train_transform, val_transform = get_transforms(IMG_SIZE)
    
    train_dataset = DeepfakeDataset(train_frames, train_labels, train_transform)
    val_dataset = DeepfakeDataset(val_frames, val_labels, val_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                              num_workers=4, pin_memory=True, prefetch_factor=3, persistent_workers=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                            num_workers=4, pin_memory=True, prefetch_factor=3, persistent_workers=True)
    
    models = {
        'MicroArtifactCNN': MicroArtifactCNN(dropout=0.5),
        'XceptionDeepfake': XceptionDeepfake(dropout=0.5),
        'ResNet50Deepfake': ResNet50Deepfake(dropout=0.5),
        'EfficientNetB0': EfficientNetB0Deepfake(dropout=0.5)
    }
    
    results = {}
    
    for model_name, model in models.items():
        print(f"\n{'='*70}")
        print(f" Training {model_name}")
        print(f"{'='*70}")
        
        model = model.to(device)
        train_losses, val_losses, train_accs, val_accs, best_acc = train_model(
            model, train_loader, val_loader, 
            epochs=EPOCHS, 
            lr=1e-4,
            model_name=model_name
        )
        
        results[model_name] = {
            'best_val_acc': best_acc,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accs': train_accs,
            'val_accs': val_accs
        }
        
        print(f"\n {model_name} completed - Best Val Acc: {best_acc:.4f}")
    
    print(f"\n{'='*70}")
    print("TRAINING SUMMARY")
    print(f"{'='*70}")
    for model_name, res in results.items():
        print(f"{model_name}: Best Val Acc = {res['best_val_acc']:.4f}")
    
    print("\n All models saved as .pth files (PyTorch format)")
    print(" Models are ready for fine-tuning and ensemble inference")


if __name__ == "__main__":
    main()

Using device: cuda
GPU: Tesla P100-PCIE-16GB
VRAM: 17.06 GB
P100 GPU fully optimized for maximum throughput
Searching for dataset directories...
Found REAL directory: /kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_original sequences
Found FAKE directory: /kaggle/input/deep-fake-detection-dfd-entire-original-dataset/DFD_manipulated_sequences/DFD_manipulated_sequences
 Initializing Haar Cascade face detector...
Haar Cascade detector initialized
  Training on Person IDs: 01, 02, 03, 04, 05, 06, 07, 08, 09, 10

Dataset Statistics:
   Total videos in REAL dir: 363
   Total videos in FAKE dir: 3068
  Filtered REAL videos (Persons 01, 02, 03, 04, 05, 06, 07, 08, 09, 10): 128
  Filtered FAKE videos (Persons 01, 02, 03, 04, 05, 06, 07, 08, 09, 10): 1224

 Extracting frames from videos using Haar Cascade...


Processing FAKE videos: 100%|██████████| 1224/1224 [3:30:22<00:00, 10.31s/it] 



Extracted 103397 total frames

 DATA BALANCING ACTIVE
   Before: Real=9473, Fake=93924 (Imbalance ratio: 9.91:1)
   Balancing to: 9473 samples per class...
   After: Real=9473, Fake=9473 (Perfect 1:1 balance)

 Training MicroArtifactCNN


Epoch 1/50 [Train]: 100%|██████████| 69/69 [01:26<00:00,  1.25s/it, loss=0.6901, acc=0.5115]



Epoch 1: Train Loss=0.7047, Train Acc=0.5115 | Val Loss=0.6930, Val Acc=0.5032
Saved best model: MicroArtifactCNN_best.pth (Val Acc: 0.5032)


...


Epoch 48: Train Loss=0.6882, Train Acc=0.5475 | Val Loss=0.6845, Val Acc=0.5533


Epoch 49/50 [Train]: 100%|██████████| 69/69 [00:19<00:00,  3.54it/s, loss=0.6912, acc=0.5463]



Epoch 49: Train Loss=0.6882, Train Acc=0.5463 | Val Loss=0.6839, Val Acc=0.5485


Epoch 50/50 [Train]: 100%|██████████| 69/69 [00:19<00:00,  3.58it/s, loss=0.6973, acc=0.5488]



Epoch 50: Train Loss=0.6882, Train Acc=0.5488 | Val Loss=0.6840, Val Acc=0.5525

 EfficientNetB0 completed - Best Val Acc: 0.5554

TRAINING SUMMARY
MicroArtifactCNN: Best Val Acc = 0.5470
XceptionDeepfake: Best Val Acc = 0.5778
ResNet50Deepfake: Best Val Acc = 0.8884
EfficientNetB0: Best Val Acc = 0.5554

 All models saved as .pth files (PyTorch format)
 Models are ready for fine-tuning and ensemble inference
